# Tarea 02: idf

### En esta tarea, se trabajará con un corpus de textos para obtener las palabras mas y menos relevantes a partir de un análisis por tf-idf. Deberás seguir los pasos de este notebook

In [29]:
"""
Paso 1) 
Del archivo adjunto (Questions.csv), extraer las primeras 2,000 preguntas presentes en la columna "Title", 
y generar un nuevo archivo que SOLO contenga dichas preguntas.
NOTA: El archivo es muy grande, por lo que deberás de procesarlo en tu equipo local (No usar colab), y 
generar con ayuda de un script el nuevo archivo, si llegas a tener problemas con el tipo de codificación
y se generan caracteres raros, deberás resolverlo con la codificación adecuada de lectura

Paso 2) 
Traducirlas las preguntas del nuevo archivo al idioma Español (Investiga cómo se realiza el proceso de 
traducción automático utilizando Google Sheets y Google Translate)
Se deberá de agregar una nueva columna al archivo .csv llamada "Textos_traducidos" donde se incluirán las 
traducciones de los textos originales (Incluir este nuevo .csv en la entrega de tu tarea)

Paso 3) 
Con este nuevo .csv de 2,000 textos traducidos al español, crear un dataset (de 2,000 textos) y aplicar 
OBLIGATORIAMENTE los siguientes preprocesamientos:
- Lematización de todas las palabras
- Filtrado de StopWords
- Pasar todo a minúsculas

Paso 4) 
Después de haber limpiado el dataset anterior, generar el vector de idf correspondiente a TODOS los textos
y mostrarlo en pantalla

Paso 5) Regresar al Dataset original, y remover todas aquellas palabras que contengan un valor de idf menor
al promedio de TODOS los idfs de la tabla obtenida

Paso 6)
Generar nuevamente la tabla de idf a partir de valores de idf para los textos filtrados y mostrarla en pantalla

Paso 7) Imprimir en pantalla el top de las 10 palabras MAS relevantes, y el top de las 20 palabras MENOS 
relevantes

NOTA: Recuerda que deberás de entregar 2 archivos, un .csv con los 2,000 textos originales y sus traducciones
y un .ipynb con todo el procedimiento realizado con sus respectivos comentarios, y DocStrings
IMPORTANTE: Todo el proceso deberá realizarse por medio de métodos, NO se aceptará programación estructurada,
por lo que, por ejemplo, deberá haber un método para filtrar StopWords, otro para obtener el promedio de 
idf de todo el conjunto de palabras, etc. 
"""

'\nPaso 1) \nDel archivo adjunto (Questions.csv), extraer las primeras 2,000 preguntas presentes en la columna "Title", \ny generar un nuevo archivo que SOLO contenga dichas preguntas.\nNOTA: El archivo es muy grande, por lo que deberás de procesarlo en tu equipo local (No usar colab), y \ngenerar con ayuda de un script el nuevo archivo, si llegas a tener problemas con el tipo de codificación\ny se generan caracteres raros, deberás resolverlo con la codificación adecuada de lectura\n\nPaso 2) \nTraducirlas las preguntas del nuevo archivo al idioma Español (Investiga cómo se realiza el proceso de \ntraducción automático utilizando Google Sheets y Google Translate)\nSe deberá de agregar una nueva columna al archivo .csv llamada "Textos_traducidos" donde se incluirán las \ntraducciones de los textos originales (Incluir este nuevo .csv en la entrega de tu tarea)\n\nPaso 3) \nCon este nuevo .csv de 2,000 textos traducidos al español, crear un dataset (de 2,000 textos) y aplicar \nOBLIGATORI

In [30]:
# Recuerda que todos los métodos que utilices deberán de contar con el formato DocString
# como en el ejemplo que se muestra a continuación:
# Ejemplo de formato Docstrings:

# def NombreFuncion(arg1, arg2, arg3):
#     """
#     Este método sirve para... utilizando... y devuelve...
    
#     Args:
#         string arg1: Esta es una cadena de texto que...
#         int arg 2: Es un número entero que se usa para...
#         dict arg 3: Diccionario que sirve para...

#     Returns:
#         string: Cadena del texto ya corregido...
#         int: El la cantidad de correcciones realizadas...
#     """

#     # Aquí debe de ir la lógica de la función (Después de la documentación)
#     Texto = ""
#     corr = 5
    
#   return Texto, corr

In [31]:
import pandas as pd
import nltk
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import Counter
import numpy as np
import re

nltk.download("punkt")
nltk.download("wordnet")
nltk.download("stopwords")

[nltk_data] Downloading package punkt to /home/eubgo/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/eubgo/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /home/eubgo/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [32]:
def _clean_char_tokens(text):
    """
    Removes unwanted characters from the tokenized words in the text.

    Parameters:
    - text (str): The tokenized text to be cleaned.

    Returns:
    - str: A string with cleaned tokens.
    """
    words = nltk.word_tokenize(text)
    cleaned_tokens = [re.sub(r"^\W+|\W*$", "", word) for word in words if len(word) > 1]
    return " ".join(cleaned_tokens)


class TextAnalysis:
    def __init__(self, filepath, flag):
        """
        Initialize the TextAnalysis object.

        Parameters:
        - filepath (str): Path to the input CSV file containing 'Translated_Texts' column.
        - flag (bool): Flag to determine whether to remove words with below-average IDF values. 
                       If set to True, the words are kept; if False, they are removed.
        """
        self.df_above_avg = None
        self.df_sorted = None
        self.average_idf = None
        self.idf_values = None
        self.df = None
        self.filepath = filepath
        self.lemmatizer = WordNetLemmatizer()
        self.stop_words = nltk.corpus.stopwords.words("spanish")

        self._initialize_analysis(flag)

    def _initialize_analysis(self, flag):
        """
        A helper function to sequentially execute the steps required for the text analysis.
        
        Parameters:
        - flag (bool): Flag to determine if below-average IDF words need to be removed.
        """
        self._read_data()
        self._preprocess_translated_text()
        self._calculate_idf()

        if not flag:
            self._remove_below_average_words_from_sentences()

    def _read_data(self):
        """
        Read the CSV data from the provided filepath and saves the 'Translated_Texts' column into a dataframe.
        """
        self.df = pd.read_csv(self.filepath)["Translated_Texts"].dropna()

    def _process_text(self, text):
        """
        Process a given text - tokenizes, lemmatizes, and removes stopwords.
        
        Parameters:
        - text (str): The input text to be processed.

        Returns:
        - str: A string with processed text.
        """
        words = nltk.word_tokenize(text)
        words = [
            self.lemmatizer.lemmatize(word.lower())
            for word in words
            if word.lower() not in self.stop_words and not (any(c.isalpha() for c in word) and any(c.isdigit() for c in word))
        ]
        return " ".join(words)

    def _preprocess_translated_text(self):
        """
        Apply the text processing and token cleaning functions to the dataframe.
        """
        self.df = self.df.apply(self._process_text).apply(_clean_char_tokens).dropna()

    def _calculate_idf(self):
        """
        Calculate the Inverse Document Frequency (IDF) values of words present in the dataset. 
        Initializes the idf_values, average_idf, and df_sorted attributes of the class.
        """
        vectorizer = TfidfVectorizer(use_idf=True)
        vectorizer.fit_transform(self.df)
        self.idf_values = pd.DataFrame({
            "Word": vectorizer.get_feature_names_out(),
            "IDF": vectorizer.idf_
        })
        self.average_idf = self.idf_values["IDF"].mean()
        self.df_sorted = self.idf_values.sort_values(by='IDF', ascending=False)

    def _filter_below_average_idf(self):
        """
        Filter out words with IDF values below the average IDF value.

        Returns:
        - list: A list of words with below-average IDF values.
        """
        below_average = self.idf_values[self.idf_values["IDF"] < self.average_idf]
        return below_average["Word"].str.lower().tolist()

    def _remove_below_average_words_from_sentences(self):
        """
        Removes words with below average IDF values from the sentences in the dataframe.
        """
        below_average_words = self._filter_below_average_idf()
        self.df_above_avg = self.df.apply(lambda text: " ".join([word for word in text.split() if word.lower() not in below_average_words]))
        series_idf_table(self.df_above_avg)


def series_idf_table(series):
    """
    Calculate and display the IDF for each word in the given pandas Series.

    Parameters:
    - series (Series): A pandas Series where each entry is a document.

    Prints:
    - Initial and filtered IDF tables showing top and bottom IDF values.
    """
    print("Filtered IDF Table:")
    tokenized_docs = series.apply(lambda x: x.split() if isinstance(x, str) else [])
    doc_freq = Counter(word for doc in tokenized_docs for word in set(doc))
    n = len(tokenized_docs)

    idf_values = {word: np.log(n / count) for word, count in doc_freq.items()}
    result = pd.Series(idf_values).sort_values(ascending=False)
    df = result.reset_index()
    df.columns = ['Word', 'IDF']
    print(df)

    display_top_bottom_idf(result)


def display_top_bottom_idf(data, top=10, bottom=10):
    """
    Prints the top and bottom words based on their IDF values.

    Parameters:
    - data (Series): A pandas Series containing IDF values indexed by words.
    - top (int, optional): Number of top words to display. Default is 10.
    - bottom (int, optional): Number of bottom words to display. Default is 10.

    Prints:
    - Top and bottom words based on IDF values.
    """
    print("\nTop {} words:".format(top))
    print("Word".ljust(20), "IDF Value")
    print("-" * 30)
    for word, idf in data.head(top).items():
        print(word.ljust(20), idf)

    print("\nBottom {} words:".format(bottom))
    print("Word".ljust(20), "IDF Value")
    print("-" * 30)
    for word, idf in data.tail(bottom).items():
        print(word.ljust(20), idf)


In [33]:
def extract_and_save_titles(input_file, output_file, num_rows=2000):
    """
    Extract a specified number of rows from the "Title" column of a CSV file and save to a new CSV file.

    Parameters:
        input_file (str): Path to the input CSV file.
        output_file (str): Path to save the extracted titles as a new CSV file.
        num_rows (int, optional): Number of rows to extract from the "Title" column. Default is 2000.
    """
    df = pd.read_csv(input_file, encoding="latin-1")
    df_titles = pd.DataFrame(df.head(num_rows)["Title"])
    df_titles.to_csv(output_file, index=False, encoding="latin-1")

In [34]:
%%capture captura
if __name__ == "__main__":
    """
    Paso 1)
    Del archivo adjunto (Questions.csv), extraer las primeras 2,000 preguntas presentes en la columna "Title",
    y generar un nuevo archivo que SOLO contenga dichas preguntas.
    NOTA: El archivo es muy grande, por lo que deberás de procesarlo en tu equipo local (No usar colab), y
    generar con ayuda de un script el nuevo archivo, si llegas a tener problemas con el tipo de codificación
    y se generan caracteres raros, deberás resolverlo con la codificación adecuada de lectura
    """
    extract_and_save_titles("Questions.csv", "Questions_2000.csv")

    """
    Paso 2) 
    Traducirlas las preguntas del nuevo archivo al idioma Español (Investiga cómo se realiza el proceso de 
    traducción automático utilizando Google Sheets y Google Translate)
    Se deberá de agregar una nueva columna al archivo .csv llamada "Textos_traducidos" donde se incluirán las 
    traducciones de los textos originales (Incluir este nuevo .csv en la entrega de tu tarea)
    """
    # In TranslatedQuestions.csv file

    """
    Paso 3) 
    Con este nuevo .csv de 2,000 textos traducidos al español, crear un dataset (de 2,000 textos) y aplicar 
    OBLIGATORIAMENTE los siguientes preprocesamientos:
    - Lematización de todas las palabras
    - Filtrado de StopWords
    - Pasar todo a minúsculas
    """
    analysis = TextAnalysis("TranslatedQuestions.csv", 1)

    """
    Paso 4) 
    Después de haber limpiado el dataset anterior, generar el vector de idf correspondiente a TODOS los textos
    y mostrarlo en pantalla
    """
    print("Initial IDF Table:")
    print(analysis.df_sorted)
    
    """
    Paso 5) Regresar al Dataset original, y remover todas aquellas palabras que contengan un valor de idf menor
    al promedio de TODOS los idfs de la tabla obtenida
    """
    """
    Paso 6)
    Generar nuevamente la tabla de idf a partir de valores de idf para los textos filtrados y mostrarla en pantalla
    """
    """
    Paso 7) Imprimir en pantalla el top de las 10 palabras MAS relevantes, y el top de las 20 palabras MENOS 
    relevantes
    """
    new_anaylsis = TextAnalysis("TranslatedQuestions.csv", 0)
    

In [35]:
with open("output.txt", "w") as f:
    f.write(captura.stdout)